<a href="https://colab.research.google.com/github/analyticsariel/projects/blob/master/StreetEasy_Get_Median_Asking_Price_from_Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streeteasy - Get Median Asking Price from Web

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com                                                      |
| External References   | Streeteasy Dashboard |
| Input Datasets        | URL links                                                                                    |
| Output Datasets       | Table of joined datasets |
| Input Data Source     | String |
| Output Data Source    | Dataframe |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 30th Sep 2021 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive

## Useful Resources
- [Google Collab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Streeteasy Data Dashboard](https://streeteasy.com/blog/data-dashboard/)

## <font color="blue">Functions</font>

In [ ]:
def read_zip(url):
  # open url
  resp = urlopen(url)
  # read contents from url file
  zipfile = ZipFile(BytesIO(resp.read()))
  # convert to dataframe
  return pd.read_csv(zipfile.open(zipfile.namelist()[0]))

In [ ]:
def melt_dataframe(df, val_name):
  # melt dataframe
  return pd.melt(df, 
                 id_vars=list(df.columns[:3]), 
                 value_vars=list(df.iloc[:, 3:].columns),# most recent date only [df.columns[-1]],
                 var_name='month', 
                 value_name=val_name)

## <font color="blue">Imports</font>

In [ ]:
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
import pandas as pd
import plotly.express as px

## <font color="blue">Transformations</font>

In [ ]:
#######################
#       SET URLS      #
#######################
se_url_dict = {}
se_url_dict['sfr'] = 'https://streeteasy-market-data-download.s3.amazonaws.com/sales/Sfr/medianAskingPrice_Sfr.zip'
se_url_dict['condo'] = 'https://streeteasy-market-data-download.s3.amazonaws.com/sales/Condo/medianAskingPrice_Condo.zip'
se_url_dict['coop'] = 'https://streeteasy-market-data-download.s3.amazonaws.com/sales/Coop/medianAskingPrice_Coop.zip'

In [ ]:
#######################
#  READ FILE PER URL  #
#######################

# create empty list for dataframes
df_list = []

# iterate through urls in dictionary
for k, v in se_url_dict.items():
  print('Get data for:', k)
  df = read_zip(v) # read value
  val_name = k + '_median_asking_price' # set value name
  df_melt = melt_dataframe(df, val_name) # melt dataframe
  df_list.append(df_melt) # append to master list of dataframes

Get data for: sfr
Get data for: condo
Get data for: coop


In [ ]:
#######################
#    COMBINE DFS      #
#######################

# combine all dataframes
df_all = df_list[0]
for i in range(1, len(df_list)):
  last_col = list(df_list[i].columns)[-1]
  df_all[last_col] = df_list[i].iloc[:, -1]

In [ ]:
print('Len data:', len(df_all))
df_all.head(10)

Len data: 27720


,areaName,Borough,areaType,month,sfr_median_asking_price,condo_median_asking_price,coop_median_asking_price
0,All Downtown,Manhattan,submarket,2010-01,7150000.0,1679000.0,749000.0
1,All Midtown,Manhattan,submarket,2010-01,4525000.0,1195000.0,595000.0
2,All Upper East Side,Manhattan,submarket,2010-01,9947500.0,1585000.0,900500.0
3,All Upper Manhattan,Manhattan,submarket,2010-01,1292500.0,636900.0,349000.0
4,All Upper West Side,Manhattan,submarket,2010-01,4122500.0,1495000.0,729023.0
5,Astoria,Queens,neighborhood,2010-01,947500.0,469000.0,NaN
6,Auburndale,Queens,neighborhood,2010-01,NaN,NaN,NaN
7,Bath Beach,Brooklyn,neighborhood,2010-01,NaN,NaN,NaN
8,Battery Park City,Manhattan,neighborhood,2010-01,NaN,875000.0,NaN
9,Bay Ridge,Brooklyn,neighborhood,2010-01,894000.0,NaN,272000.0


In [ ]:
# filter on borough and neighborhood
df_plot = df_all.loc[(df_all['Borough'] == 'Brooklyn') & 
                     (df_all['areaName'].isin(['Williamsburg', 'Bay Ridge', 'Flatbush']))]
df_plot.head()

,areaName,Borough,areaType,month,sfr_median_asking_price,condo_median_asking_price,coop_median_asking_price
9,Bay Ridge,Brooklyn,neighborhood,2010-01,894000.0,NaN,272000.0
68,Flatbush,Brooklyn,neighborhood,2010-01,NaN,NaN,159000.0
192,Williamsburg,Brooklyn,neighborhood,2010-01,1099000.0,625000.0,NaN
207,Bay Ridge,Brooklyn,neighborhood,2010-02,884500.0,569000.0,272000.0
266,Flatbush,Brooklyn,neighborhood,2010-02,NaN,NaN,159000.0


In [ ]:
# view line chart comparison of boroughs
fig = px.line(df_plot, x="month", y="sfr_median_asking_price", color='areaName', title='SFR Median Asking Price')
fig.show()

# End Notebook